In [1]:
import pickle

In [20]:
groupjoin = pickle.load(open("data/groupjoin","rb"))

In [21]:
len(groupjoin)

5952

In [5]:
import pandas as pd

In [6]:
groupjoin = pickle.load(open("data/groupjoin","rb"))

In [7]:
len(groupjoin)

5952

In [15]:
len(groupjoin[1671174]),groupjoin[1671174][:10]

(1737,
 [(7467415, 1278656196000),
  (10290000, 1278656973000),
  (8978228, 1278677405000),
  (9041705, 1278681464000),
  (11413753, 1278685331000),
  (11738963, 1278686299000),
  (4323799, 1278688849000),
  (6657235, 1278697584000),
  (9061588, 1278698802000),
  (6008990, 1278699541000)])

In [16]:
ersvp = pickle.load(open("data/ersvp","rb"))

In [19]:
len(ersvp),ersvp['dlfrldysgbbc']

(294571,
 [(7918822, 1397910371000),
  (103016352, 1397757003000),
  (104586062, 1397918738000)])

In [20]:
new_grp_event = pickle.load(open("data/new_grp_event","rb"))

In [27]:
len(new_grp_event),len(new_grp_event[1671174]),new_grp_event[1671174][:10]

(5647,
 177,
 [('14013449', 1279029600000),
  ('14097177', 1282917600000),
  ('14704584', 1292598000000),
  ('16638356', 1298484000000),
  ('16908526', 1307111400000),
  ('24160391', 1310135400000),
  ('57096322', 1332608400000),
  ('56590142', 1335535200000),
  ('62400822', 1338559200000),
  ('67315922', 1340978400000)])

In [30]:
x = []
for i in new_grp_event :
    if i in groupjoin:
        x.append(i)

In [49]:
def GetValidGrps(new_grp_event,groupjoin) :
    return [i for i in new_grp_event if i in groupjoin]
            

In [36]:
len(x),x[:10]

(5319,
 [1671174,
  1638409,
  15859722,
  1769484,
  18645005,
  18448408,
  1703963,
  20185123,
  20447271,
  16711722])

In [50]:
len(GetValidGrps(new_grp_event,groupjoin))

5319

In [45]:
def make_social_graph_of_grp_members(grp_id,groupjoin,ersvp,new_grp_event) :
    member_set = set()
    for member,time in groupjoin[grp_id] :
        member_set.add(member)
    
    events_in_grp = []
    
    for event_id,time in new_grp_event[grp_id] :
        if event_id in ersvp:
            events_in_grp.append(event_id)
    edges = {}
    for event_id in events_in_grp :

        event_member_set = list(set([member for member,time in ersvp[event_id]  if member in member_set ]))
        event_member_set = sorted(event_member_set)
        for i in range(len(event_member_set)) :
            for j in range(i+1,len(event_member_set)):
                if (event_member_set[i],event_member_set[j]) in edges :
                    edges[(event_member_set[i],event_member_set[j])]+=1
                else :
                    edges[(event_member_set[i],event_member_set[j])]=1
    return list(member_set),[(i,j,edges[(i,j)]) for i,j in edges]
                
        
        
        
    
    

In [46]:
members,edges =  make_social_graph_of_grp_members(1671174,groupjoin,ersvp,new_grp_event)

In [47]:
(len(members),members[:10]),(len(edges),edges[:10])

((1737,
  [4677635,
   78503952,
   72146962,
   211894314,
   181026862,
   192176181,
   190709814,
   7921723,
   182566981,
   209363031]),
 (55199,
  [(5620833, 6657235, 1),
   (5620833, 7467415, 1),
   (5620833, 8978228, 2),
   (5620833, 9041705, 3),
   (5620833, 9061588, 1),
   (5620833, 9420414, 2),
   (5620833, 10290000, 1),
   (5620833, 11350459, 2),
   (5620833, 11413753, 2),
   (5620833, 11738963, 5)]))

In [48]:
def make_member_level_features(grp_id,groupjoin,ersvp,new_grp_event) :
    members,edges =  make_social_graph_of_grp_members(grp_id,groupjoin,ersvp,new_grp_event)
    adjacency_list = {}
    for member in members :
        adjacency_list[member] = []
    for i,j,w in edges :
        adjacency_list[i].append((j,w))
        adjacency_list[j].append((i,w))
    

In [51]:
def get_memberId_to_groups(valid_grps,groupjoin) :
    memberId_to_groups = {}
    for grp in valid_grps :
        for member , time in groupjoin[grp] :
            if member in  memberId_to_groups :
                memberId_to_groups[member].append(grp)
            else :
                memberId_to_groups[member] = [grp]
    return memberId_to_groups
    
    

In [54]:
def get_grp_to_memberId_to_attended_events(grp_to_valid_events,groupjoin,ersvp):
    grp_to_memberId_to_attended_events = {}
    for grp in grp_to_valid_events :
        memberId_to_attended_events = {}
        for event,_ in grp_to_valid_events[grp] :
            for member,time in ersvp[event] :
                if member in memberId_to_attended_events :
                    memberId_to_attended_events.append(event)
                else :
                    memberId_to_attended_events = [event]
        grp_to_memberId_to_attended_events[grp] = memberId_to_attended_events
    return grp_to_memberId_to_attended_events
                
    
    
    

In [55]:
def get_grp_to_valid_events(valid_grps,new_grp_event,ersvp):
    grp_to_valid_events = {}
    for i in valid_grps :
        for j,_ in new_grp_event[i] :
            if j in ersvp :
                grp_to_valid_events[i].append(j)
    return grp_to_valid_events

In [57]:
from sklearn.decomposition import NMF

In [58]:
mat = [[2,3,5,6],[45,89,4,36]]

In [64]:
model = NMF(n_components =3 )

In [65]:
model.fit(mat)

/home/sanket/miniconda3/envs/data_science/lib/python3.7/site-packages/sklearn/decomposition/_nmf.py:315: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  "'nndsvda' in 1.1 (renaming of 0.26)."), FutureWarning)


NMF(n_components=3)

In [66]:
nmf_features = model.transform(mat)

In [67]:
nmf_features

array([[ 0.22759124,  0.73157561,  0.77159923],
       [ 8.60254867, 14.46340326,  0.        ]])

In [68]:
model.components_

array([[1.32996650e+00, 7.40893980e+00, 3.06939420e-03, 2.87414106e+00],
       [2.31995587e+00, 1.74713467e+00, 2.74681609e-01, 7.79672477e-01],
       [0.00000000e+00, 4.61814381e-02, 6.21817075e+00, 6.18961015e+00]])

In [69]:
import pickle

In [72]:
elat = pickle.load(open("data/elat","rb"))
elon = pickle.load(open("data/elon","rb"))

In [73]:
len(elat),len(elon)

(458087, 458087)

In [75]:
list(elat.keys())[:10]

['dvlgvlywjbxb',
 'dlfrldysgbbc',
 'hrkzllyvfbjc',
 '170374282',
 '228592712',
 'njvctlyvlbhc',
 'dpqbgcyrnbqb',
 'dxzpffywjbfc',
 'dlzvbfyrcbtb',
 '161926702']

In [77]:
elat['dvlgvlywjbxb'],elon['dvlgvlywjbxb']

(41.908699, -87.6819)

In [78]:
from pathlib import Path

In [100]:
len([str(x).split('/')[-1].split('.')[0] for x in Path("member_level_features").iterdir()])

5320

In [88]:
[x for x in Path("member_level_features").iterdir()][:10]

[PosixPath('member_level_features/1671174.csv'),
 PosixPath('member_level_features/1638409.csv'),
 PosixPath('member_level_features/15859722.csv'),
 PosixPath('member_level_features/1769484.csv'),
 PosixPath('member_level_features/18645005.csv'),
 PosixPath('member_level_features/18448408.csv'),
 PosixPath('member_level_features/1703963.csv'),
 PosixPath('member_level_features/20185123.csv'),
 PosixPath('member_level_features/20447271.csv'),
 PosixPath('member_level_features/16711722.csv')]

In [89]:
df = pd.read_csv("member_level_features/1671174.csv")

In [90]:
df

,member,group_id,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12
0,4677635,1671174,72,1,13,0.918296,3.700440,1.584963,374.916667,33.944444,13.013889,1.724832,2.668147,2.092105
1,78503952,1671174,0,0,23,0.000000,4.175736,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,72146962,1671174,0,0,11,1.000000,3.459432,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,211894314,1671174,0,0,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,181026862,1671174,0,0,6,1.000000,2.584963,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1732,15802341,1671174,60,1,3,0.000000,1.584963,0.000000,388.300000,38.966667,10.266667,1.659389,2.472409,2.008393
1733,134496232,1671174,81,1,2,0.000000,1.000000,0.000000,338.407407,28.444444,14.123457,1.566956,2.668083,2.028720
1734,201498614,1671174,0,0,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1735,62038012,1671174,0,0,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [91]:
x = {}

In [92]:
x[(45,89)] =456

In [93]:
x[(1,-1)] =12

In [95]:
x[(1,-0)]

KeyError: (1, 0)

In [96]:
groupjoin = pickle.load(open("data/groupjoin","rb"))
new_grp_event = pickle.load(open("data/new_grp_event","rb"))
ersvp = pickle.load(open("data/ersvp","rb"))

In [97]:
len(groupjoin)

5952

In [98]:
len(new_grp_event)

5647

In [99]:
len(ersvp)

294571